## 关于Python的面试题，整理自GitHub两个关于Python面试的仓库1-2

### 1. is 和 == 的区别
is 比较的是内存地址是否一致
== 比较的是变量的内容是否一致

Python为了优化性能，在-6-256的数值范围内嵌了一个链表，如果整型变量的取值在该范围内  Python不会重新设置新的对象（Python一切皆对象），参照下文的代码

In [14]:
a = 257
b = 257
print(a == b) #返回1，因为a,b变量的内容，数值相等
print(a is b) #返回0，两个变量指向不同的内存地址


c = 55
d = 55
print(c == d) #返回1，因为a,b变量的内容，数值相等
print(c is d) #返回1,见上文的原因，指向同一内存地址

True
False
True
True


还需要注意的是，Python语言在同一代码块下，赋相同值的不同变量指向同一内存地址  
同一代码块：一个模块，一个函数体，一个类，交互式（Jupyter）的一行代码

In [15]:
c = 257
def main():
    a = 257
    b = 257
    print(a is b) #返回1，原因如上
    print(a is c) #返回0
if __name__ == '__main__':
    main()

True
False


## 2. 嵌套列表的坑

- Pyhton的列表为一种容器，可以用来承载其他的对象，同时也可以承载容器对象，下方的代码b其实是对内层列表容器对象的引用，他记录了对象在内存中的地址  
- 通过访问该内存地址，可以访问到对象，当我们使用乘法的方式来构建嵌套列表时,实际是将[0]\*3这个内部列表的地址进行了5次复制，一旦更改了某个值
- 所有的五个容器内的元素的值都会一起更改

In [16]:
b = [[0] * 3] * 5
for i in b:
    print(id(i)) #使用id函数查看内部列表的地址，发现均是同一个地址

b[0][0] = 100
for i in b:
    print(i) #更改了某一元素时，所有内部容器的值都发生了变更

2700065850568
2700065850568
2700065850568
2700065850568
2700065850568
[100, 0, 0]
[100, 0, 0]
[100, 0, 0]
[100, 0, 0]
[100, 0, 0]


- 正确的列表嵌套写法

In [17]:
b = [[0] * 3 for _ in range(5)]
for i in b:
    print(id(i)) #每一个内部列表都指向了不同的地址，引用了不同的对象

2700065013704
2700065124424
2700064280776
2700065010824
2700065578504


## 3. 高阶函数 Filter/Map/Reduce,匿名函数lambda

In [29]:
'''map函数接受一个函数和一个可迭代对象为参数，将函数作用于可迭代对象的每一个元素上
返回一个迭代器
匿名函数lambda可以不指定函数的名称，语法为lambda 变量：表达式'''
print(map(abs,[(-1)**i*i for i in range(10)])) #返回一个迭代器，可迭代对象:filter
print(list(map(abs,[(-1)**i*i for i in range(10)])))
print(list(map(lambda x:x**2,[i for i in range(5)]))) #传入匿名函数作为参数

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 1, 4, 9, 16]


In [19]:
'''reduce函数将一个数据集合（链表，元组等）中的所有数据
进行下列操作：用传给 reduce 中的函数 function（有两个参数）
先对集合中的第 1、2 个元素进行操作，得到的结果
再与第三个数据用 function 函数运算，最后得到一个结果。
'''
from functools import reduce
print(reduce(lambda x,y:x*y,[1,2,3,4,5])) #reduce内部传入的参数必须含有两个参数

120


In [24]:
'''filter函数用于过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表。
filter接收两个参数，第一个为函数，第二个为序列，
序列的每个元素作为参数传递给函数进行判断，
然后返回 True 或 False，最后将返回 True 的元素放到新列表中。
调用方法filter(function, iterable)，第二个参数可以是列表，也可以是懒列表'''
list(filter(lambda x:x>0,[-1.-4,5,6])) #返回一个filter对象，也是一个可迭代对象(懒惰列表)
#传入的lambda函数返回1或0，其意义就是，筛选出列表中大于0的元素

[5, 6]

## 4. \*args 和 \*\*args以及不指定个数的可迭代对象解压方法

In [110]:
'''f函数的两个参数均为可变参数，即参数的个数时不固定的，
前者为可变位置参数，后者为可变关键字参数(两者顺序不可以交换)'''
def f(*args,**kargs): 
    print(args)
    print(kargs)
f([1,2,3,5],a=5,b=6)

([1, 2, 3, 5],)
{'a': 5, 'b': 6}


In [111]:
'''关键字参数需要放置两个*号，下面的函数接收个数不定的可变参数（变量名=值）
在函数调用时，会自动将其组织为键值对的形式'''
def f1(name,**kw):
    print('name:',name,'other:',kw)
f1('tom',gender='male',cup = 'd')   

name: tom other: {'gender': 'male', 'cup': 'd'}


In [33]:
'''命名关键字参数:限制关键字参数的名称'''
def f2(name,age,*,gender,cup):
    print('name:',name,'age:',age,gender,cup)
f2('kizaki', 18, gender='male', cup='D')
f2('akari', 20, gender='male', city='tokoyo')

name: kizaki age: 18 male D


TypeError: f2() got an unexpected keyword argument 'city'

In [27]:
# 调用函数时，也可以使用*，**对传入参数进行解压缩,将字典解压成关键字 键=值
def f(a,b,c):
    print(f'{a}+{b}={c}')
f(**{'a':15,'b':16,'c':31})

def f1(a,b,c):
    print(f'{a}+{b}={c}')
f1(*[15,16,31])

15+16=31
15+16=31


## 5. 闭包，变量的作用域
在一个内部函数中，对外部作用域的变量进行引用，(并且一般外部函数的返回值为内部函数)  
内部函数就被认为是闭包  
变量的作用域：变量的作用域总是在代码中变量被赋值的地方所决定  
变量的搜索路径：Local-Enclosing locals-Global-builtin  
* Local即局部空间，函数体内部定义的名称（包括函数参数）
* Enclosing：非全局，也非局部的
* Global是全局空间
* builtin:即函数内部自带的名称，如abs函数等

In [1]:
def x():
    pass
print(x)
def x():
    pass
print(x)

<function x at 0x000001462ED6BA68>
<function x at 0x000001462ED6B0D8>


In [6]:
# 一个简单的闭包函数的例子
def f1(a):
    def f2():
        return a*a
    return f2 #返回函数，而不是返回函数的调用
print(f1(5)) #调用外部函数，返回内函数的引用(地址)
print(f1(4) == f1(4)) #返回值为函数f2,每次外部调用都会创建一个新的对象(引用不同的地址)
print(f1==f1) #返回True，外部函数f1之定义了一次，只有一个地址
f1(5)() #调用内部函数

<function f1.<locals>.f2 at 0x0000019661B17C18>
False
True


25

In [4]:
'''
1. Python的问题就在于，当循环结束以后，循环体中的临时变量i不会销毁，
   而是继续存在于执行环境中。
2. 还有一个python的现象是，
   python的函数只有在执行时，才会去找函数体里的变量的值。
3. 可能有些人认为这段代码的执行结果应该是2,3,4.但是实际的结果是4,4,4。
   loop在python中是没有域的概念的，flist在像列表中添加func的时候， 
   并没有保存i的值，而是当执行f(2)的时候才去取， 
   这时候循环已经结束，i的值是2，所以结果都是4''' 
flist = []  
for i in range(3):   
    def foo(x):
        print(x + i) 
    flist.append(foo)
for f in flist:   
    f(2)

4
4
4


# 6. Python的垃圾回收机制
[Python垃圾回收](https://sutune.me/2018/10/14/python-GC/)

# 7. 变量的进阶理解
[变量、引用（函数参数）、可变与不可变类型、变量的作用域](https://www.cnblogs.com/guigujun/p/9926446.html)

# 8. Python中的元类
[深入理解Python元类](https://www.cnblogs.com/JetpropelledSnake/p/9094103.html)

# 9. 格式化字符串的三种写法

In [23]:
salary = 15000
print(f'我想要的工资数额为：{salary:.1f}')
print('我想要的工资数额为：%s'%salary)

#format使用方法为在原始字符串中用{idx}占位
print('我想要的工资数额为：{0:.2f}'.format(salary))

我想要的工资数额为：15000.0
我想要的工资数额为：15000
我想要的工资数额为：15000.00


# 10. 迭代器和生成器

本节代码示例中，list1为可迭代对象，可以使用for循环进行迭代遍历  
可以在list1上迭代n次，因为其中的每个元素都是存储在内存上的  

将列表生成式的[]转变为(),或者在函数中使用yield关键词以产生生成器  
生成器对象只有每次对其进行迭代时，才会生成所对应的一个值，  
并且在生成该值后，下一次迭代会在内存中丢弃上一个值，因此如  
(列表生成式)这种类型的迭代器只能进行一次迭代  
(某些情况下可以起到节省内存空间的作用)，而函数式迭代器可以进行  
无数次迭代，因为每次调用函数都会重新创造出该迭代器对象  


In [18]:
list1 = [i for i in range(3)]
print(type(list1))
print(type(iter(list1))) #列表是一个可迭代对象，但是不是迭代器，调用iter()后构建为迭代器
gen1 = (i for i in range(3))
print(gen1) #generator对象，生成器
for i in gen1:
    print(i)
for i in gen1: #只能进行一次迭代遍历
    print(i)

<class 'list'>
<class 'list_iterator'>
<generator object <genexpr> at 0x000001980147DA48>
0
1
2


In [126]:
def f():
    for i in range(3):
        yield i
print(f()) #调用f函数，f()为一个生成器对象

#迭代遍历得到第i个元素
for i in f():
    print(i)

<generator object f at 0x00000207973D1BC8>
0
1
2


# 11. 装饰器、面向切面编程

[装饰器高阶](https://stackoverflow.com/questions/739654/how-to-make-function-decorators-and-chain-them-together)    
* 进阶写法包括链式装饰器，装饰器装饰器的装饰器
* 面向切面编程AOP实际上就是指装饰器,其应用于插入日志，性能测试，事务处理等方面
* 为已经存在的对象(函数，类的方法）增加新的功能

# 12. 鸭子类型
“当看到一只鸟走起来像鸭子、游泳起来像鸭子、叫起来也像鸭子，那么这只鸟就可以被称为鸭子。”

我们并不关心对象是什么类型，到底是不是鸭子，只关心行为。

比如在python中，有很多file-like的东西，比如StringIO,GzipFile,socket。它们有很多相同的方法，我们把它们当作文件使用。

又比如list.extend()方法中,我们并不关心它的参数是不是list,只要它是可迭代的,所以它的参数可以是list/tuple/dict/字符串/生成器等.

鸭子类型在动态语言中经常使用，非常灵活，使得python不像java那样专门去弄一大堆的设计模式。

# 13.函数重载
函数重载（英语：function overloading），是Ada、C++、C#、D和Java等编程语言中具有的一项特性，这项特性允许创建数项名称相同但输入输出类型或个数不同的子程序，它可以简单地称为一个单独功能可以执行多项任务的能力。

# 14. read,readline,readlines的区别
* read 读取整个文件
* readline 读取下一行,使用生成器方法
* readlines 读取整个文件到一个迭代器以供我们遍历

# 15. super().\__init\__语句的含义，用法

# 16. 单例模式的实现